In [ ]:
!pip install opencv-python
!pip install pytesseract

In [ ]:
!sudo apt install tesseract-ocr

In [ ]:
!unzip data.zip -d data

In [ ]:
import cv2
import pytesseract
import glob
import os

# mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

path = "/trial-imgs/*.*"

img_num = 0
for file in glob.glob(path):

  # get filename of page with extension removed
  pg_fn = os.path.basename(file)
  pg_fn = os.path.splitext(pg_fn)[0]

  # read image from which text needs to be extracted
  img = cv2.imread(file)

  # preprocessing the image starts

  # convert the image to gray scale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


  # performing OTSU threshold
  ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)


  # specify structure shape and kernel size.
  # kernel size increases or decreases the area
  # of the rectangle to be detected.
  # smaller value like (10, 10) will detect
  # each word instead of a sentence.
  rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))

  # applying dilation on the threshold image
  dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

  # finding contours
  contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                          cv2.CHAIN_APPROX_NONE)

  # creating copy of image
  im2 = img.copy()

  # looping through the identified contours
  # then rectangular part is cropped and passed on
  # to pytesseract for extracting text from it
  # extracted text is then written into the text file
  i = 0
  for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    # cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]

    # apply OCR on the cropped image to create annotations
    text = pytesseract.image_to_string(cropped)
    
    pic_name = pg_fn + '-crop-' + str(i) + ".jpg"
    if w >= 100 and h >= 100:
      imge = cv2.imwrite(pic_name, cropped)
      i +=1
    
    # # drawing a rectangle around boundingRect and annotating it for reference
    # rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 1)
    
    # anno_name = 'ref' + str(img_num) + '.jpg'
    # anno_rect = cv2.putText(rect, text[:30] + str(w) + "x" + str(h), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 1)
    # status = cv2.imwrite(anno_name, rect)

    print(text)
  img_num += 1